# Convolutional Neural Network (CNN) - 2 Examples with Keras:

**Source (all credits to):** *TensorFlow Documentation*

1. First example = begginer tutorial
2. Second example = advanced tutorial

# 1. Import Libraries

In [1]:
# !pip install -U tensorflow-addons

# !pip install -q "tqdm>=4.36.1"

import tensorflow as tf
import tensorflow_addons as tfa

# progress bar for training/test
import tqdm

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [3]:
print("We're using TensorFlow", tf.__version__)
print("We're using Keras", tf.keras.__version__)
print("We're using TQDM", tqdm.__version__)

We're using TensorFlow 2.4.1
We're using Keras 2.4.0
We're using TQDM 4.47.0


# 2. Prepare Data

## 2.1. Preprocessing Data

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# normalize data
x_train, x_test = x_train / 255.0, x_test / 255.0

## 2.2. View Data

In [6]:
print('Shapes')
print('---')
print('X_train =', x_train.shape)
print('y_train =', y_train.shape)
print('X_test =', x_test.shape)
print('y_test =', y_test.shape)

Shapes
---
X_train = (60000, 28, 28)
y_train = (60000,)
X_test = (10000, 28, 28)
y_test = (10000,)


# 3. Example 1 - CNN Basics

In [11]:
# build the model using the Sequential API
model = Sequential()                          # it is a feed-forward network without loops like in RNN
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

# look at all layers and parameter count
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# now we "compile" the model specifying the loss and optimizer
model.compile(
    loss='categorical_crossentropy', # this is our cross-entropy
    optimizer='adam',
    metrics=['accuracy']             # report accuracy during training
)

# load callback to progress bar in training
tqdm_callback = tfa.callbacks.TQDMProgressBar()


# and now we can fit the model with model.fit()
# and we don't have to write loops and batching manually as in TensorFlow
model.fit(
    X_train_flat, 
    y_train_oh,
    batch_size=512, 
    epochs=10,
    validation_data=(X_val_flat, y_val_oh),
    callbacks=[tqdm_callback],
    verbose=0
)

# for validation/test
y_pred = model.predict_classes(X_test_flat)

In [10]:
# initialize tqdm callback with default parameters
tqdm_callback = tfa.callbacks.TQDMProgressBar()

# train the model with tqdm_callback
# make sure to set verbose = 0 to disable
# the default progress bar.
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          verbose=0,
          callbacks=[tqdm_callback],
          validation_data=(x_test, y_test))

print('---')
print('TQDMProgressBar also works with evaluate:')

# TQDMProgressBar() also works with evaluate()
model.evaluate(x_test, y_test, batch_size=64, callbacks=[tqdm_callback], verbose=0)

Epoch 1/10



Epoch 2/10



Epoch 3/10



Epoch 4/10



Epoch 5/10


...


Epoch 8/10



Epoch 9/10



Epoch 10/10




---
TQDMProgressBar also works with evaluate:


[0.0866035670042038, 0.9796000123023987]

# 4. Example 2 - CNN Advanced

## 4.1. Load Libraries

In [13]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## 4.2. Prepare Data

In [15]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# shuffle the data
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)



## 4.3. Build Model Using Keras Subclassing

In [17]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# create an instance of the model
model = MyModel()

In [18]:
# choose the loss and the optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

# choose metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

############# define training step function #############
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

############# define test step function #############
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

############# training and test CNN #############    

EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )
    

Epoch 1, Loss: 0.13665059208869934, Accuracy: 95.88666534423828, Test Loss: 0.05534274876117706, Test Accuracy: 98.23999786376953
Epoch 2, Loss: 0.04566776007413864, Accuracy: 98.6066665649414, Test Loss: 0.056701574474573135, Test Accuracy: 98.07999420166016
Epoch 3, Loss: 0.023973630741238594, Accuracy: 99.22333526611328, Test Loss: 0.05403752624988556, Test Accuracy: 98.31999969482422
Epoch 4, Loss: 0.014527365565299988, Accuracy: 99.54000091552734, Test Loss: 0.07906388491392136, Test Accuracy: 97.89999389648438
Epoch 5, Loss: 0.010260012000799179, Accuracy: 99.66999816894531, Test Loss: 0.06954068690538406, Test Accuracy: 98.22000122070312
